In [1]:
import sys
import cv2
import numpy as np
from PyQt5.QtWidgets import (QApplication, QMainWindow, QWidget, QVBoxLayout, QHBoxLayout, 
                             QLabel, QPushButton, QFileDialog, QSlider, QStatusBar, QMessageBox)
from PyQt5.QtGui import QImage, QPixmap
from PyQt5.QtCore import Qt, QTimer

class FaceBlurApp(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Flouter les visages")
        self.setGeometry(100, 100, 1200, 600)
        self.image = None
        self.blurred_image = None
        self.original_backup = None  # sauvegarde de l'image originale
        self.capture = None
        self.timer = QTimer()
        self.init_ui()

    def init_ui(self):
        # Widget principal
        main_widget = QWidget()
        self.setCentralWidget(main_widget)
        main_layout = QVBoxLayout()
        main_widget.setLayout(main_layout)

        # Style sombre
        self.setStyleSheet("""
            QMainWindow { background-color: #1f2937; }
            QLabel { color: #f3f4f6; font-size: 14px; }
            QPushButton { 
                background-color: #2563eb; 
                color: white; 
                padding: 8px 16px; 
                border-radius: 5px; 
                font-weight: bold; 
            }
            QPushButton:hover { background-color: #1d4ed8; }
            QPushButton:disabled { background-color: #6b7280; }
            QSlider::groove:horizontal {
                border: 1px solid #4b5563;
                height: 8px;
                background: #4b5563;
                border-radius: 4px;
            }
            QSlider::handle:horizontal {
                background: #2563eb;
                border: 1px solid #1d4ed8;
                width: 18px;
                margin: -2px 0;
                border-radius: 9px;
            }
        """)

        # Titre
        title_label = QLabel("Flouter les visages dans une image")
        title_label.setAlignment(Qt.AlignCenter)
        title_label.setStyleSheet("font-size: 24px; font-weight: bold; color: #f3f4f6; margin-bottom: 16px;")
        main_layout.addWidget(title_label)

        # Curseur pour l'intensité du flou
        blur_layout = QHBoxLayout()
        blur_label = QLabel("Intensité du flou :")
        self.blur_value_label = QLabel("99")
        self.blur_slider = QSlider(Qt.Horizontal)
        self.blur_slider.setMinimum(9)
        self.blur_slider.setMaximum(99)
        self.blur_slider.setValue(99)
        self.blur_slider.setSingleStep(2)
        self.blur_slider.valueChanged.connect(self.update_blur_value)
        blur_layout.addWidget(blur_label)
        blur_layout.addWidget(self.blur_slider)
        blur_layout.addWidget(self.blur_value_label)
        main_layout.addLayout(blur_layout)

        # Zone d'affichage des images
        image_layout = QHBoxLayout()
        self.original_image_display = QLabel()
        self.original_image_display.setAlignment(Qt.AlignCenter)
        self.original_image_display.setMinimumSize(400, 300)
        self.original_image_display.setStyleSheet("border: 2px solid #4b5563; border-radius: 8px;")
        image_layout.addWidget(self.original_image_display)

        self.blurred_image_display = QLabel()
        self.blurred_image_display.setAlignment(Qt.AlignCenter)
        self.blurred_image_display.setMinimumSize(400, 300)
        self.blurred_image_display.setStyleSheet("border: 2px solid #4b5563; border-radius: 8px;")
        image_layout.addWidget(self.blurred_image_display)
        main_layout.addLayout(image_layout)

        # Boutons
        button_layout = QHBoxLayout()
        self.load_button = QPushButton("Charger une image")
        self.load_button.clicked.connect(self.load_image)
        button_layout.addWidget(self.load_button)

        self.process_button = QPushButton("Flouter les visages")
        self.process_button.clicked.connect(self.process_image)
        self.process_button.setEnabled(False)
        button_layout.addWidget(self.process_button)

        self.save_button = QPushButton("Sauvegarder l'image")
        self.save_button.clicked.connect(self.save_image)
        self.save_button.setEnabled(False)
        button_layout.addWidget(self.save_button)

        self.restore_button = QPushButton("Restaurer l'image")
        self.restore_button.clicked.connect(self.restore_original_image)
        self.restore_button.setEnabled(False)
        button_layout.addWidget(self.restore_button)

        self.camera_button = QPushButton("Activer la caméra")
        self.camera_button.clicked.connect(self.toggle_camera)
        button_layout.addWidget(self.camera_button)

        main_layout.addLayout(button_layout)

        # Barre de statut
        self.status_bar = QStatusBar()
        self.status_bar.setStyleSheet("color: #f3f4f6;")
        self.setStatusBar(self.status_bar)
        self.status_bar.showMessage("Veuillez charger une image.")

    def update_blur_value(self):
        self.blur_value_label.setText(str(self.blur_slider.value()))

    def load_image(self):
        file_name, _ = QFileDialog.getOpenFileName(self, "Charger une image", "", 
                                                   "Images (*.png *.jpg *.jpeg *.bmp)")
        if file_name:
            self.image = cv2.imread(file_name)
            self.original_backup = self.image.copy()
            if self.image is None:
                QMessageBox.critical(self, "Erreur", "Impossible de charger l'image.")
                return
            self.display_image(self.image, self.original_image_display)
            self.blurred_image = None
            self.blurred_image_display.clear()
            self.process_button.setEnabled(True)
            self.save_button.setEnabled(False)
            self.restore_button.setEnabled(False)
            self.status_bar.showMessage("Image chargée. Cliquez sur 'Flouter les visages'.")

    def process_image(self):
        if self.image is None:
            return

        self.status_bar.showMessage("Traitement en cours...")
        self.process_button.setEnabled(False)
        QApplication.processEvents()

        try:
            cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_alt2.xml")
            gray = cv2.cvtColor(self.image, cv2.COLOR_BGR2GRAY)
            faces = cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=6, minSize=(60, 60))
            self.blurred_image = self.image.copy()
            blur_size = self.blur_slider.value()
            

            for (x, y, w, h) in faces:
                face_region = self.blurred_image[y:y+h, x:x+w]
                face_region_blurred = cv2.GaussianBlur(face_region, (blur_size, blur_size), 20)
                self.blurred_image[y:y+h, x:x+w] = face_region_blurred

            self.display_image(self.blurred_image, self.blurred_image_display)
            self.save_button.setEnabled(True)
            self.restore_button.setEnabled(True)
            self.status_bar.showMessage(f"Traitement terminé. {len(faces)} visage(s) détecté(s).")
        except Exception as e:
            QMessageBox.critical(self, "Erreur", str(e))
            self.status_bar.showMessage("Erreur lors du traitement.")
        finally:
            self.process_button.setEnabled(True)

    def restore_original_image(self):
        if self.original_backup is not None:
            self.image = self.original_backup.copy()
            self.display_image(self.image, self.original_image_display)
            self.blurred_image_display.clear()
            self.status_bar.showMessage("Image restaurée.")

    def save_image(self):
        if self.blurred_image is None:
            return

        file_name, _ = QFileDialog.getSaveFileName(self, "Sauvegarder l'image", "", 
                                                   "Images (*.png *.jpg *.jpeg)")
        if file_name:
            cv2.imwrite(file_name, self.blurred_image)
            self.status_bar.showMessage(f"Image sauvegardée : {file_name}")

    def toggle_camera(self):
        if self.capture is None:
            self.capture = cv2.VideoCapture(0)
            self.timer.timeout.connect(self.update_camera_frame)
            self.timer.start(30)
            self.camera_button.setText("Désactiver la caméra")
        else:
            self.timer.stop()
            self.capture.release()
            self.capture = None
            self.camera_button.setText("Activer la caméra")
            self.original_image_display.clear()

    def update_camera_frame(self):
        ret, frame = self.capture.read()
        if not ret:
            return

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_alt2.xml")
        faces = cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=6, minSize=(60, 60))
        blur_size = self.blur_slider.value()

        for (x, y, w, h) in faces:
            roi = frame[y:y+h, x:x+w]
            roi_blurred = cv2.GaussianBlur(roi, (blur_size, blur_size), 30)
            frame[y:y+h, x:x+w] = roi_blurred

        self.display_image(frame, self.original_image_display)

    def display_image(self, image, label):
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        h, w, ch = image_rgb.shape
        bytes_per_line = ch * w
        q_image = QImage(image_rgb.data, w, h, bytes_per_line, QImage.Format_RGB888)
        pixmap = QPixmap.fromImage(q_image)
        pixmap = pixmap.scaled(label.size(), Qt.KeepAspectRatio, Qt.SmoothTransformation)
        label.setPixmap(pixmap)

if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = FaceBlurApp()
    window.show()
    sys.exit(app.exec_())
    print(faces.shape)


SystemExit: 0

c:\Users\21625\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
